In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
import tqdm
import datetime
import os
import pickle
import random

from glicko2 import Player

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [ ]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [ ]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [ ]:
players_df = dfs['players_df']
players_df.head()

In [ ]:
ranking_df = dfs['ranking_df']
ranking_df.head()

In [ ]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

In [ ]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data)")

sets_df.head()

In [ ]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

In [ ]:
# Code optimization by Dan
# Basically we want to replace this line in process_tournament with something more efficient:
#
#      tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
#
# Instead, we can
# - Merge the tournament date info into ``sets_df``
# - Sort by date
# - Store the start/end positions of each tournament in a separate dictionary
# - Use tournament_sets_df = sets_df.iloc[start:end+1] instead.

sets_df = sets_df.merge(tournament_info_df[['key', 'start', 'end']], left_on='tournament_key', right_on='key', how='left')
sets_df = sets_df.drop(labels=['key_y'], axis='columns')
sets_df = sets_df.rename(columns={"key_x": "key"})
sets_df = sets_df.sort_values(by=['end', 'tournament_key']) # Just in case there are tournaments with the exact same end date

In [ ]:
# A bit of data cleanup
min_date = datetime.datetime(2015, 1, 1)
max_date = datetime.datetime(2024, 12, 31)

sets_df = sets_df[(sets_df['start'] >= min_date) & (sets_df['end'] >= min_date) & (sets_df['start'] <= max_date) & (sets_df['end'] <= max_date)]

## Generate features

In [ ]:
# Append to this list evey time you add features!
# For convenience, it is blank here, and you should append in each feature-adding cell
features = []
output = 'winner'

In [ ]:
sets_df[output] = sets_df.apply(lambda row: 1.0 if row['winner_id'] == row['p1_id'] else 0.0, axis=1)

### "Default" ELO/RD features

In [ ]:
# Overall ELO/RD as features
player_ratings_df = pd.read_pickle(data_path + 'overall_players_ranking_new_weekly.pkl')
player_rds_df = pd.read_pickle(data_path + 'overall_players_rds_new_weekly.pkl')

# Might as well compute a (mostly accurate) number of updates from the ratings dataframe
# This isn't 100% accurate, as sometimes (especially if default elo playing default elo)
# the elo might not change from round to round.
player_updates_df = pd.DataFrame(0, index=player_ratings_df.index, columns=player_ratings_df.columns)

for i in range(1, len(player_ratings_df.index)):
    row_updates = (player_ratings_df.iloc[i] != player_ratings_df.iloc[i-1]).astype(int)
    player_updates_df.iloc[i] = row_updates

player_updates_df = player_updates_df.cumsum()


features_elo_rd = ['p1_elo', 'p2_elo', 'p1_rd', 'p2_rd', 'p1_updates', 'p2_updates']
dates = list(player_ratings_df.index)

# Efficiency purposes. We are assuming that constant intervals are used.
initial_date = dates[0]
interval = dates[1] - dates[0]

# start_date is the start date of the specific set (tournament)
def get_info(df, player_id, start_date):
    # Far more efficient than searching the index manually every time
    week_num = int((start_date - initial_date) / interval)
    newest_date = initial_date + week_num * interval
    if dates[-1] < newest_date:
        newest_date = dates[-1]
    return df.loc[newest_date, player_id]

sets_df['p1_elo'] = sets_df.apply(lambda row: get_info(player_ratings_df, row['p1_id'], row['start']), axis=1)
sets_df['p2_elo'] = sets_df.apply(lambda row: get_info(player_ratings_df, row['p2_id'], row['start']), axis=1)

sets_df['p1_rd'] = sets_df.apply(lambda row: get_info(player_rds_df, row['p1_id'], row['start']), axis=1)
sets_df['p2_rd'] = sets_df.apply(lambda row: get_info(player_rds_df, row['p2_id'], row['start']), axis=1)

sets_df['p1_updates'] = sets_df.apply(lambda row: get_info(player_updates_df, row['p1_id'], row['start']), axis=1)
sets_df['p2_updates'] = sets_df.apply(lambda row: get_info(player_updates_df, row['p2_id'], row['start']), axis=1)

# Might as well clear if these aren't needed. They are huge in memory.
del player_ratings_df
del player_rds_df
del player_updates_df

### Number of times each player has played each character

In [ ]:
initial_date = datetime.datetime(2015,1,1)
interval = datetime.timedelta(weeks=1)

end_date = initial_date + math.ceil((sets_df['end'].max() - initial_date) / interval) * interval

# Assumes game_data_extractor.ipynb was run
game_data = pd.read_pickle(data_path + 'individual_game_data.pkl')

In [ ]:
# Actually get a list of characters and their individual popularities
# In the end, a series with index as the character and value as the number of times played.
# Probably sorted in the end.
all_characters = pd.concat([game_data['winner_char'], game_data['loser_char']], ignore_index=True)
all_characters = all_characters.value_counts()

In [ ]:
# Player/character combos
game_data['winner_pc'] = game_data.apply(lambda row: row['winner_id'] + '/' + row['winner_char'], axis=1)
game_data['loser_pc'] = game_data.apply(lambda row: row['loser_id'] + '/' + row['loser_char'], axis=1)

# What week number this is, for example.
game_data['end_index'] = game_data['end'].apply(lambda x: math.ceil((x - initial_date) / interval))
game_data['end_date'] = game_data['end_index'].apply(lambda x: initial_date + x*interval)
game_data['end_date_copy'] = game_data['end_index'].apply(lambda x: initial_date + x*interval) # To deal with include_groups=True deprecation


In [ ]:
# Actually compute the dates (index) without loading a dataframe for reference.
dates = []

date = initial_date
while date <= end_date:
    dates.append(date)
    date += interval

pc_combos = list(set(list(game_data['winner_pc']) + list(game_data['loser_pc'])))

# Count the number of times a character has been used.
# Columns will be player/character (or rather, player/melee/character)
# Initialize everything to zero, initially.
character_usage_df = pd.DataFrame(0, index=dates, columns=pc_combos)

In [ ]:
def update_row(df):
    date = df.iloc[0]['end_date_copy']

    pc_series = pd.concat([df['winner_pc'], df['loser_pc']], ignore_index=True)

    # Deprecation-proofing
    temp = character_usage_df.loc[date].copy()
    temp.update(pc_series.value_counts())
    
    character_usage_df.loc[date] = temp
    
game_data.groupby('end_date').apply(update_row, include_groups=False)

In [ ]:
character_usage_df = character_usage_df.cumsum()
character_usage_df

In [ ]:
# Newest possible date to get data from.
sets_df['start_index'] = sets_df['start'].apply(lambda x: int((x - initial_date) / interval))
sets_df['start_date'] = sets_df['start_index'].apply(lambda x: initial_date + x * interval)

In [ ]:
# No point in recomputing every single time.
p1c_final_index = ['p1' + '_' + x + '_count' for x in all_characters.index]
p2c_final_index = ['p2' + '_' + x + '_count' for x in all_characters.index]

features_pc = p1c_final_index + p2c_final_index

def get_char_count(row):
    # We'll rename the index entries later for these.
    p1c_series = pd.Series(0, index=[row['p1_id'] + '/' + x for x in all_characters.index])
    p2c_series = pd.Series(0, index=[row['p2_id'] + '/' + x for x in all_characters.index])

    p1c_series.update(character_usage_df.loc[row['start_date']])
    p2c_series.update(character_usage_df.loc[row['start_date']])

    p1c_series.index = p1c_final_index
    p2c_series.index = p2c_final_index

    return pd.concat([p1c_series, p2c_series])

sets_df = pd.concat([sets_df, sets_df.apply(get_char_count, axis=1)], axis=1)

## Compute most-used characters for each individual player (mains)

In [ ]:
p1_char_usage_df = sets_df[p1c_final_index]
p2_char_usage_df = sets_df[p2c_final_index]

p1_char_usage_df

In [ ]:
def get_mains(row, player):
    # Compute three mains and their usages
    index = ['m1', 'm2', 'm1_usage', 'm2_usage']
    index = [player + '_' + x for x in index]

    row_sorted = row.sort_values(ascending=False)

    mains = []

    # First two entries are the actual mains
    for i in range(0,2):
        mains.append(row_sorted.index[i].removeprefix(player + '_').removesuffix('_count'))

    # Next two are how many times they have been played
    for i in range(0,2):
        mains.append(row_sorted.iloc[i])

    return pd.Series(mains, index=index)


p1_mains_df = p1_char_usage_df.apply(lambda row: get_mains(row, 'p1'), axis=1)
p2_mains_df = p2_char_usage_df.apply(lambda row: get_mains(row, 'p2'), axis=1)

In [ ]:
# Temporary workspace for computing individual matchup elos
player_mains_df = pd.concat([sets_df[['p1_id', 'p2_id', 'start']], p1_mains_df, p2_mains_df], axis=1)

# TODO: Compute this properly!
start_date = datetime.datetime(2015,1,1)
interval = datetime.timedelta(weeks=1)

# Newest possible date to pull matchup info from
player_mains_df['start_index'] = player_mains_df['start'].apply(lambda x: int((x - start_date) / interval))
player_mains_df['newest_date'] = player_mains_df['start_index'].apply(lambda x: start_date + x*interval)

player_mains_df

In [ ]:
features_mains = ['p1_m1_usage', 'p1_m2_usage', 'p2_m1_usage', 'p2_m2_usage']

## player/char/char elos and updates, alt2 variant

In [ ]:
alt2_df = pd.read_pickle(data_path + 'char_vs_char_player_rankings_weekly_alt2.pkl')

# How many times each column was updated so far
# Keep everything float to not change the data type
alt2_updates_df = alt2_df.copy()
alt2_updates_df.iloc[0] = 0.0
alt2_updates_df.iloc[1:] = (alt2_updates_df.iloc[1:].reset_index(drop=True) != alt2_updates_df.iloc[:-1].reset_index(drop=True)).astype(float).values
alt2_updates_df = alt2_updates_df.cumsum()

start_date = alt2_df.index[0]
interval = alt2_df.index[1] - alt2_df.index[0]

In [ ]:
# TODO: Pull number of times this specific matchup happened as well?

matchups = []

# Compute the actual matchups first.
# Then obtain the elos.

for i in [1,2]: # player_i
    for j in [1,2]: # main_j
        for k in [1,2]: # opponent_main_k
            player_num = i
            opponent_num = 3-i # swap 1 and 2

            player = player_mains_df['p' + str(player_num) + '_id']
            player_main = player_mains_df['p' + str(player_num) + '_m' + str(j)]
            opponent_main = player_mains_df['p' + str(opponent_num) + '_m' + str(k)]

            col_name = 'p' + str(player_num) + '/m' + str(j) + '/m' + str(k)

            matchups.append(col_name)
            player_mains_df[col_name] = player + '/' + player_main + '/' + opponent_main

player_mains_df


In [ ]:
# NOTE: This cell in particular likes to crash if you interrupt it.
#       Do NOT interrupt it.

def get_entry(row, matchup, df, default_value):
    if row[matchup] not in df.columns:
        return default_value
    else:
        return df.at[row['newest_date'], row[matchup]]

# p1/m1/m1, etc...
for matchup in tqdm.tqdm(matchups):
    player_mains_df[matchup + '_elo'] = player_mains_df[['newest_date', matchup]].apply(lambda row: get_entry(row, matchup, alt2_df, 1500.0), axis=1)
    features_mains.append(matchup + '_elo')

    player_mains_df[matchup + '_updates'] = player_mains_df[['newest_date', matchup]].apply(lambda row: get_entry(row, matchup, alt2_updates_df, 0.0), axis=1)
    features_mains.append(matchup + '_updates')

In [ ]:
# Save memory
del alt2_df
del alt2_updates_df

In [ ]:
player_mains_df

## player/char/char elos and updates, alt

In [ ]:
alt_df = pd.read_pickle(data_path + 'char_vs_char_player_rankings_weekly_alt.pkl')

# How many times each column was updated so far
# Keep everything float to not change the data type
alt_updates_df = alt_df.copy()
alt_updates_df.iloc[0] = 0.0
alt_updates_df.iloc[1:] = (alt_updates_df.iloc[1:].reset_index(drop=True) != alt_updates_df.iloc[:-1].reset_index(drop=True)).astype(float).values
alt_updates_df = alt_updates_df.cumsum()

start_date = alt_df.index[0]
interval = alt_df.index[1] - alt_df.index[0]

In [ ]:
# TODO: Pull number of times this specific matchup happened as well?

matchups = []
reference_matchups = [] # player/same_char/same_char to compare against

# Compute the actual matchups first.
# Then obtain the elos.

for i in [1,2]: # player_i
    for j in [1,2]: # main_j
        for k in [1,2]: # opponent_main_k
            player_num = i
            opponent_num = 3-i # swap 1 and 2

            player = player_mains_df['p' + str(player_num) + '_id']
            player_main = player_mains_df['p' + str(player_num) + '_m' + str(j)]
            opponent_main = player_mains_df['p' + str(opponent_num) + '_m' + str(k)]

            col_name = 'p' + str(player_num) + '/m' + str(j) + '/m' + str(k)

            matchups.append(col_name)
            player_mains_df[col_name] = player + '/' + player_main + '/' + opponent_main

# opponent/omain/omain as a reference for player/pmain/omain
for i in [1,2]: # player_i
    for j in [1,2]: # main_j
        opponent_num = 3-i # swap 1 and 2

        opponent = player_mains_df['p' + str(opponent_num) + '_id']
        opponent_main = player_mains_df['p' + str(opponent_num) + '_m' + str(k)]

        col_name = 'p' + str(opponent_num) + '/m' + str(j) + '/m' + str(j)
        
        reference_matchups.append(col_name)
        player_mains_df[col_name + '_ref'] = opponent + '/' + opponent_main + '/' + opponent_main 

player_mains_df


In [ ]:
# NOTE: This cell in particular likes to crash if you interrupt it.
#       Do NOT interrupt it.

def get_entry(row, matchup, df, default_value):
    if row[matchup] not in df.columns:
        return default_value
    else:
        return df.at[row['newest_date'], row[matchup]]

# p1/m1/m1, etc...
for matchup in tqdm.tqdm(matchups):
    player_mains_df[matchup + '_alt_elo'] = player_mains_df[['newest_date', matchup]].apply(lambda row: get_entry(row, matchup, alt_df, 1500.0), axis=1)
    features_mains.append(matchup + '_alt_elo')

    player_mains_df[matchup + '_alt_updates'] = player_mains_df[['newest_date', matchup]].apply(lambda row: get_entry(row, matchup, alt_updates_df, 0.0), axis=1)
    features_mains.append(matchup + '_alt_updates')

# p1/same_m1/same_m1, etc...
for reference_matchup in tqdm.tqdm(reference_matchups):
    player_mains_df[reference_matchup + '_alt_ref_elo'] = player_mains_df[['newest_date', reference_matchup + '_ref']].apply(lambda row: get_entry(row, reference_matchup + '_ref', alt_df, 1500.0), axis=1)
    features_mains.append(reference_matchup + '_alt_ref_elo')

    player_mains_df[reference_matchup + '_alt_ref_updates'] = player_mains_df[['newest_date', reference_matchup + '_ref']].apply(lambda row: get_entry(row, reference_matchup + '_ref', alt_updates_df, 0.0), axis=1)
    features_mains.append(reference_matchup + '_alt_ref_updates')

In [ ]:
# Save memory
del alt_df
del alt_updates_df

In [ ]:
player_mains_df

## player/char "global" elos (also called "alt3")

In [ ]:
pc_rankings_df = pd.read_pickle(data_path + 'player_char_overall_rankings_weekly.pkl')
pc_rds_df = pd.read_pickle(data_path + 'player_char_overall_rds_weekly.pkl')

pc_rankings_updates_df = pc_rankings_df.copy()
pc_rankings_updates_df.iloc[0] = 0.0
pc_rankings_updates_df.iloc[1:] = (pc_rankings_df.iloc[1:].reset_index(drop=True) != pc_rankings_df.iloc[:-1].reset_index(drop=True)).astype(float).values
pc_rankings_updates_df = pc_rankings_updates_df.cumsum()

pc_rankings_updates_df

In [ ]:
pc_combos = []

# Compute the actual matchups first.
# Then obtain the elos.

for i in [1,2]: # player_i
    for j in [1,2]: # main_j
        player_num = i

        player = player_mains_df['p' + str(player_num) + '_id']
        player_main = player_mains_df['p' + str(player_num) + '_m' + str(j)]

        col_name = 'p' + str(player_num) + '/m' + str(j)

        pc_combos.append(col_name)
        player_mains_df[col_name] = player + '/' + player_main

player_mains_df

In [ ]:
def get_entry(row, matchup, df, default_value):
    if row[matchup] not in df.columns:
        return default_value
    else:
        return df.at[row['newest_date'], row[matchup]]

# p1/m1/m1, etc...
for pc_combo in tqdm.tqdm(pc_combos):
    player_mains_df[pc_combo + '_elo'] = player_mains_df[['newest_date', pc_combo]].apply(lambda row: get_entry(row, pc_combo, pc_rankings_df, 1500.0), axis=1)
    features_mains.append(pc_combo + '_elo')

    player_mains_df[pc_combo + '_rd'] = player_mains_df[['newest_date', pc_combo]].apply(lambda row: get_entry(row, pc_combo, pc_rds_df, 350.0), axis=1)
    features_mains.append(pc_combo + '_rd')

    player_mains_df[pc_combo + '_updates'] = player_mains_df[['newest_date', pc_combo]].apply(lambda row: get_entry(row, pc_combo, pc_rankings_updates_df, 0.0), axis=1)
    features_mains.append(pc_combo + '_updates')

In [ ]:
player_mains_df

In [ ]:
# Save memory
del pc_rankings_df
del pc_rankings_updates_df
del pc_rds_df

## Generate final dataset from above features

In [ ]:
dataset_df = pd.concat([sets_df[features_elo_rd], player_mains_df[features_mains], sets_df[[output]]], axis=1)
dataset_df = dataset_df.astype(float)
dataset_df

In [ ]:
dataset_df.to_pickle(data_path + 'dataset_mini.pkl')